In [1]:
from vpython import *
import random
import itertools
import csv


<IPython.core.display.Javascript object>

In [2]:
scene = canvas(title='Tre corpi',x=0, y=0, width=1500, height=1500,center=vector(0,0,0))
vss = scene

<IPython.core.display.Javascript object>

In [3]:
# pos è una tupla di valori che definisce la posizione
# mas un valore reale >0
# color è un colore 
# velocity è una tupla che definisce la velocità iniziale
def create_body(pos,mass,radius,color,velocity,trail):
    body = sphere()
    body.pos = vector(pos[0],pos[1],pos[2])
    body.radius =radius
    body.color = color
    body.mass = mass
    body.v = vector(velocity[0], velocity[1],velocity[2])
    body.p=body.v*body.mass
    body.trail=trail
    return body
    
def get_random_position(x,y,z):
    return(random.uniform(x[0],x[1]),random.uniform(y[0],y[1]),random.uniform(z[0],z[1]))

def get_random_velocity(x,y,z):
    return(random.uniform(x[0],x[1]),random.uniform(y[0],y[1]),random.uniform(z[0],z[1]))
def generate_bodies(n,stars):
    for i in range(n):
      new_position=get_random_position((-1,1),(0,2),(-2,2))
      new_velocity=get_random_velocity((0,1),(0.1,0.2),(-5,5))
      stars.append(create_body(new_position,10.0,0.4,color.blue,new_velocity))
    
def read_test_data(data,stars):
    for obj in data:
        mass=float(obj[0])
        position=[float(obj[1]),float(obj[2]),float(obj[3])]
        velocity=[float(obj[4]),float(obj[5]),float(obj[6])]
        radius=float(obj[7])
        trail=obj[8]
        c=obj[9].split(';')
        obj_color=vector(float(c[0]),float(c[1]),float(c[2]))
        stars.append(create_body(position,mass,radius,obj_color,velocity,trail))
file = open('files/test_data_5.csv')
csvreader = csv.reader(file)
header=next(csvreader)
test_data=[]
for h_object in csvreader:
    test_data.append(h_object)

three_stars=[]
#generate_bodies(3,three_stars)
read_test_data(test_data,three_stars)
def get_cm_position(stars):
    cm=vector(0.0,0.0,0.0)
    total_mass=0.0
    for star in stars:
        cm+=star.mass*star.pos
        total_mass+=star.mass
    return cm/total_mass  
#la funzione translate_to_cm riferisce tutti i corpi al sistema del centro di massa
def translate_to_cm(stars,center_of_mass):
    for star in stars:
        star.pos=star.pos-center_of_mass
def get_cm_velocity(stars):
    vcm=vector(0.0,0.0,0.0)
    total_mass=0.0
    for star in stars:
        vcm+=star.mass*star.v
        total_mass+=star.mass
    return vcm/total_mass
def do_galileo_transform(stars,vcm):
    for star in stars:
        star.v=star.v-vcm

#proviamo le funzioni appena definite
print("Old cm position")
current_cm=get_cm_position(three_stars)
print(current_cm)
print("New cm position")
translate_to_cm(three_stars,current_cm)
print(get_cm_position(three_stars))
print("Old cm velocity")
current_vcm=get_cm_velocity(three_stars)
print(current_vcm)
do_galileo_transform(three_stars,current_vcm)
print("New cm velocity")
print(get_cm_velocity(three_stars))

#definisco alcune costanti per la simulazione, il passo temporale della simulazione e la costante di gravitazione universale
dt=0.001
G=1.0

star1=three_stars[0]
star2=three_stars[1]

#il vettore r_ij tra stella due e stella1
rvector = star1.pos-star2.pos
force_vector = -G * star1.mass * star2.mass * rvector / mag(rvector)**3
#costruiamo la rappresentazione dell'orbita dei nostri tre corpi

for star in three_stars:
  if (star.trail=="1"):
    star.orbit = curve(color=star.color, radius = 0.008)
#Inizializziamo le quantità di moto delle stelle

for i in  range(len(three_stars)):
    three_stars[i].p=three_stars[i].mass*three_stars[i].v
def pstep( star1, star2,delta_t):
  ## calcolo delle nuove quantità di moto per la determinazione delle nuove
  # posizioni (metodo leapfrog)
  rvector = star1.pos-star2.pos
  force_vector = -G * star1.mass * star2.mass * rvector / mag(rvector)**3
  star1.p = star1.p + force_vector*delta_t
  star2.p = star2.p - force_vector*delta_t
    
#inizializzo il calcolo
for pair in itertools.combinations(three_stars,2):
    pstep(pair[0],pair[1],0.5*dt)

#istante iniziale della simlazione    
time=0

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Old cm position
<6.66644e-06, 1.66661e-05, 0>
New cm position
<3.62665e-17, -1.8476e-18, 0>
Old cm velocity
<-6.66644e-06, -1.66661e-05, 0>
New cm velocity
<-1.7393e-17, -2.4981e-17, 0>


In [ ]:
#eseguo il calcolo per 5 unità di tempo arbitrarie

while time<35:
  rate(50)
  time+=dt
  for pair in itertools.combinations(three_stars,2):
    pstep(pair[0],pair[1],dt)
  for star in three_stars:
    star.pos = star.pos + star.p/star.mass * dt
    #scene.center=body.pos
    if (star.trail=="1"):
        star.orbit.append(pos=star.pos)